## 幾種思路

思路1：TF-IDF + 機器學習分類器
直接使用TF-IDF對文本提取特徵，並使用分類器進行分類。在分類器的選擇上，可以使用SVM、LR、或者XGBoost。

思路2：FastText
FastText是入門款的詞向量，利用Facebook提供的FastText工具，可以快速構建出分類器。

思路3：WordVec + 深度學習分類器
WordVec是進階款的詞向量，並通過構建深度學習分類完成分類。深度學習分類的網絡結構可以選擇TextCNN、TextRNN或者BiLSTM。

思路4：Bert詞向量
Bert是高配款的詞向量，具有強大的建模學習能力。

## 獲取6種模型的資料集

In [100]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import re
path = '../data/0414/review_data(seg+pos+stopwords)_n+v+f+p.csv'
df = pd.read_csv(path)

### 檢查重複值、空值

In [101]:
#印出重複資料
print(df[df.duplicated()])

Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []


In [102]:
#移除重複值
#df = df.drop_duplicates()
#print(df.shape)

In [103]:
#印出空值資料
#df_train[df_train.isnull().T.any()]

### 切分為6個資料集

In [104]:
def split_df(df):
    df_value = df[['value','filtered_word']]
    df_value.rename(columns={'value': 'label'}, inplace=True)
    df_comfort = df[['comfort','filtered_word']]
    df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
    df_location = df[['location','filtered_word']]
    df_location.rename(columns={'location': 'label'}, inplace=True)
    df_cleanliness = df[['cleanliness','filtered_word']]
    df_cleanliness.rename(columns={'cleanliness': 'label'}, inplace=True)
    df_service = df[['service','filtered_word']]
    df_service.rename(columns={'service': 'label'}, inplace=True)
    df_facilities = df[['facilities','filtered_word']]
    df_facilities.rename(columns={'facilities': 'label'}, inplace=True)
    return df_value, df_comfort, df_location, df_cleanliness, df_service, df_facilities

In [105]:
#df_value_train, df_comfort_train, df_location_train, df_cleanliness_train, df_service_train ,df_facilities_train = split_df(df_train)
#df_value_test, df_comfort_test, df_location_test, df_cleanliness_test, df_service_test, df_facilities_test = split_df(df_test)
df_value, df_comfort, df_location, df_cleanliness, df_service ,df_facilities = split_df(df)

In [106]:
df_value

,label,filtered_word
0,1.0,"價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH..."
1,0.0,"內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(..."
2,0.0,"房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH),分離(VHC)"
3,1.0,"房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)..."
4,1.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...
1267,0.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH..."
1268,0.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,0.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,0.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH)..."


### 清理資料(移除詞性標註的文字)

In [107]:
def remove_N_comma(sentence):
    # 把後面(N..)(V..)(F..)拿掉
    sentence = str(sentence)
    pattern = re.compile(r"\([N,V,F,P].*?\)") #移除詞性標示
    sentence = re.sub(pattern, '', sentence)
    pattern = re.compile(r",") #將逗號替換為空格
    sentence = re.sub(pattern, ' ', sentence)
    return sentence
pd.options.mode.chained_assignment = None  # 忽略警告

In [108]:
#訓練集
df_value['filtered_word'] = df_value.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)

In [109]:
print(df_value.shape)
df_value

(1272, 2)


,label,filtered_word
0,1.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,0.0,房間 小 美中不足 乾 濕 分離
3,1.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,1.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,0.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,0.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,0.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,0.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


## 模型架構

### 套件引用

In [110]:
#import package
#轉向量用
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy.sparse import coo_matrix

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate
import pickle #儲存模型用
from sklearn.model_selection import train_test_split
#類別採樣
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine
from imblearn.pipeline import make_pipeline as make_pipeline_imb


#模型
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier

#模型效能表現
import sklearn.metrics as metrics

### 顯示訓練結果

In [111]:
def classification_report(y_test, pre):
    #混淆矩陣
    confusion = metrics.confusion_matrix(y_test, pre)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN)
    #Accuracy
    accuracy = (TP+TN)/float(TP+TN+FN+FP)
    print("Accuracy：", accuracy)
    #Sensitivity(Recall)
    recall = TP/float(TP+FN)
    print("Recall：", recall)
    #Specificity
    specificity = TN/float(TN+FP)
    print("Specificity：", specificity)
    #Precision
    precision = TP/float(TP+FP)
    print("Precision：", precision)
    #f1-score
    f1_score = ((2*precision*recall)/(precision+recall))
    print("f1_score：", f1_score)
    #AUC
    print("AUC：", metrics.roc_auc_score(y_test, pre))

In [112]:
df_value

,label,filtered_word
0,1.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,0.0,房間 小 美中不足 乾 濕 分離
3,1.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,1.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,0.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,0.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,0.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,0.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


### 切分訓練、測試數據

In [113]:
from sklearn.model_selection import train_test_split
def split_label(df,seed):
    X = df.filtered_word.tolist()
    y = df.label
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=1/3,random_state=seed)
    return X_train,X_test,y_train,y_test

In [114]:
X_train_1,X_test_1,y_train_1,y_test_1 = split_label(df_value,1)
X_train_2,X_test_2,y_train_2,y_test_2 = split_label(df_value,2)
X_train_3,X_test_3,y_train_3,y_test_3 = split_label(df_value,3)

In [115]:
s1 = pd.Series(y_train_1)
freq1 = s1.value_counts() 
print(freq1) 
s2 = pd.Series(y_train_2)
freq2 = s2.value_counts() 
print(freq2) 
s3 = pd.Series(y_train_3)
freq3 = s3.value_counts() 
print(freq3) 

0.0    688
1.0    160
Name: label, dtype: int64
0.0    697
1.0    151
Name: label, dtype: int64
0.0    697
1.0    151
Name: label, dtype: int64


### 模型設計

#### (1) baseline

In [116]:
def SVM_model(X_train,X_test,y_train,y_test):
    print("SVM baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [117]:
def LR_model(X_train,X_test,y_train,y_test):
    print("LR baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), LogisticRegression(random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [118]:
def RF_model(X_train,X_test,y_train,y_test):
    print("RF baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), RandomForestClassifier(max_depth=2, random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [124]:
def AdaBoost_model(X_train,X_test,y_train,y_test):
    print("AdaBoost_model")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    feature_names = model.named_steps["tfidfvectorizer"].get_feature_names()
    print(feature_names)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [125]:
AdaBoost_model(X_train_1,X_test_1,y_train_1,y_test_1)

AdaBoost_model
['00', '10', '100', '1025', '12', '1200', '14樓', '1700', '180', '1樓', '200', '201', '2160', '2250', '24', '24h', '25', '2500', '2樓', '30', '300', '400', '50', '500', '550', '5800', '5星級', '600', '711', '80', '800', '80年', '8點', '900', '9900', '9oo', '9點半', 'agoda', 'bb', 'buffet', 'cp', 'cp值', 'diy', 'freebar', 'ktv', 'nice', 'ok', 'ps', 'spa', 'wifi', 'xd', 'ㄧ些', '一下', '一些', '一千', '一半', '一品', '一家人', '一望無際', '一樓', '一次性', '一流', '一清二楚', '一級', '一級棒', '一點', '一點點', '丁香魚', '三五', '三合一', '三四', '三明治', '三樓', '三溫暖', '三義', '三龜', '上心', '上鎖', '上限', '下來', '下午茶', '下去', '下樓', '下降', '下雨天', '不佳', '不便', '不值', '不到', '不理', '不耐煩', '不良', '不行', '不足', '不遠', '不錯', '世界', '世紀', '丟臉', '中國風', '中央', '中心', '中心點', '中正', '中班', '中繼站', '中華', '中餐', '中餐廳', '丹麥', '主人', '主人家', '主動', '主題', '主題房', '久侯', '久坐', '之後', '九份', '乳豬', '乾乾淨淨', '乾凈', '乾淨', '乾濕', '乾老闆', '事項', '二樓', '二館', '五星級', '交流道', '交給', '交誼廳', '交通', '享受', '享用', '人員', '人工', '人情味', '人房', '人潮', '人間', '今天', '今日', '介紹', '介紹給', '付費', '付錢', '仙境', '仙草', '代替', '

#### (2)執行採樣 => 解決類別不平衡 (SVM)

In [79]:
def SVM_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [80]:
def SVM_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [81]:
def SVM_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [82]:
def SVM_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (3)執行採樣 => 解決類別不平衡 (Adaboost)

In [84]:
def AdaBoost_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [85]:
def AdaBoost_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [86]:
def AdaBoost_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [87]:
def AdaBoost_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

## 模型訓練&結果

### 資料集1

#### baseline

In [90]:
SVM_model(X_train_1,X_test_1,y_train_1,y_test_1)
LR_model(X_train_1,X_test_1,y_train_1,y_test_1)
RF_model(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model(X_train_1,X_test_1,y_train_1,y_test_1)

SVM baseline
TP: 37
TN: 359
FP: 2
FN: 26
Accuracy： 0.9339622641509434
Recall： 0.5873015873015873
Specificity： 0.9944598337950139
Precision： 0.9487179487179487
f1_score： 0.7254901960784315
AUC： 0.7908807105483007


LR baseline
TP: 14
TN: 361
FP: 0
FN: 49
Accuracy： 0.8844339622641509
Recall： 0.2222222222222222
Specificity： 1.0
Precision： 1.0
f1_score： 0.3636363636363636
AUC： 0.6111111111111112


RF baseline
TP: 0
TN: 361
FP: 0
FN: 63
Accuracy： 0.8514150943396226
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_24784/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 42
TN: 356
FP: 5
FN: 21
Accuracy： 0.9386792452830188
Recall： 0.6666666666666666
Specificity： 0.9861495844875346
Precision： 0.8936170212765957
f1_score： 0.7636363636363636
AUC： 0.8264081255771005




#### 處理類別不平衡(SVM)

In [83]:
SVM_model2(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model3(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model4(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 36
TN: 349
FP: 12
FN: 27
Accuracy： 0.9080188679245284
Recall： 0.5714285714285714
Specificity： 0.9667590027700831
Precision： 0.75
f1_score： 0.6486486486486486
AUC： 0.7690937870993273


SMOTE
TP: 33
TN: 351
FP: 10
FN: 30
Accuracy： 0.9056603773584906
Recall： 0.5238095238095238
Specificity： 0.9722991689750693
Precision： 0.7674418604651163
f1_score： 0.6226415094339623
AUC： 0.7480543463922965


RandomOverSampler
TP: 35
TN: 350
FP: 11
FN: 28
Accuracy： 0.9080188679245284
Recall： 0.5555555555555556
Specificity： 0.9695290858725761
Precision： 0.7608695652173914
f1_score： 0.6422018348623854
AUC： 0.7625423207140659


RandomUnderSampler
TP: 42
TN: 312
FP: 49
FN: 21
Accuracy： 0.8349056603773585
Recall： 0.6666666666666666
Specificity： 0.8642659279778393
Precision： 0.46153846153846156
f1_score： 0.5454545454545455
AUC： 0.7654662973222529




#### 處理類別不平衡(Adaboost)

In [91]:
AdaBoost_model2(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model3(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model4(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 43
TN: 338
FP: 23
FN: 20
Accuracy： 0.8985849056603774
Recall： 0.6825396825396826
Specificity： 0.9362880886426593
Precision： 0.6515151515151515
f1_score： 0.6666666666666666
AUC： 0.8094138855911709


SMOTE
TP: 42
TN: 344
FP: 17
FN: 21
Accuracy： 0.910377358490566
Recall： 0.6666666666666666
Specificity： 0.9529085872576177
Precision： 0.711864406779661
f1_score： 0.6885245901639344
AUC： 0.8097876269621421


RandomOverSampler
TP: 42
TN: 344
FP: 17
FN: 21
Accuracy： 0.910377358490566
Recall： 0.6666666666666666
Specificity： 0.9529085872576177
Precision： 0.711864406779661
f1_score： 0.6885245901639344
AUC： 0.8097876269621421


RandomUnderSampler
TP: 40
TN: 323
FP: 38
FN: 23
Accuracy： 0.8561320754716981
Recall： 0.6349206349206349
Specificity： 0.8947368421052632
Precision： 0.5128205128205128
f1_score： 0.5673758865248226
AUC： 0.7648287385129491




### 資料集2

#### baseline

In [92]:
SVM_model(X_train_2,X_test_2,y_train_2,y_test_2)
LR_model(X_train_2,X_test_2,y_train_2,y_test_2)
RF_model(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model(X_train_2,X_test_2,y_train_2,y_test_2)

SVM baseline
TP: 47
TN: 352
FP: 0
FN: 25
Accuracy： 0.9410377358490566
Recall： 0.6527777777777778
Specificity： 1.0
Precision： 1.0
f1_score： 0.7899159663865547
AUC： 0.8263888888888888


LR baseline
TP: 20
TN: 352
FP: 0
FN: 52
Accuracy： 0.8773584905660378
Recall： 0.2777777777777778
Specificity： 1.0
Precision： 1.0
f1_score： 0.4347826086956522
AUC： 0.6388888888888888


RF baseline
TP: 0
TN: 352
FP: 0
FN: 72
Accuracy： 0.8301886792452831
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_24784/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 55
TN: 350
FP: 2
FN: 17
Accuracy： 0.9551886792452831
Recall： 0.7638888888888888
Specificity： 0.9943181818181818
Precision： 0.9649122807017544
f1_score： 0.8527131782945737
AUC： 0.8791035353535352




#### 處理類別不平衡(SVM)

In [93]:
SVM_model2(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model3(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model4(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 53
TN: 342
FP: 10
FN: 19
Accuracy： 0.9316037735849056
Recall： 0.7361111111111112
Specificity： 0.9715909090909091
Precision： 0.8412698412698413
f1_score： 0.7851851851851852
AUC： 0.85385101010101


SMOTE
TP: 52
TN: 345
FP: 7
FN: 20
Accuracy： 0.9363207547169812
Recall： 0.7222222222222222
Specificity： 0.9801136363636364
Precision： 0.8813559322033898
f1_score： 0.7938931297709924
AUC： 0.8511679292929293


RandomOverSampler
TP: 52
TN: 346
FP: 6
FN: 20
Accuracy： 0.9386792452830188
Recall： 0.7222222222222222
Specificity： 0.9829545454545454
Precision： 0.896551724137931
f1_score： 0.7999999999999999
AUC： 0.8525883838383838


RandomUnderSampler
TP: 58
TN: 298
FP: 54
FN: 14
Accuracy： 0.839622641509434
Recall： 0.8055555555555556
Specificity： 0.8465909090909091
Precision： 0.5178571428571429
f1_score： 0.6304347826086957
AUC： 0.8260732323232323




#### 處理類別不平衡(Adaboost)

In [94]:
AdaBoost_model2(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model3(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model4(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 51
TN: 343
FP: 9
FN: 21
Accuracy： 0.9292452830188679
Recall： 0.7083333333333334
Specificity： 0.9744318181818182
Precision： 0.85
f1_score： 0.7727272727272727
AUC： 0.8413825757575758


SMOTE
TP: 55
TN: 345
FP: 7
FN: 17
Accuracy： 0.9433962264150944
Recall： 0.7638888888888888
Specificity： 0.9801136363636364
Precision： 0.8870967741935484
f1_score： 0.8208955223880596
AUC： 0.8720012626262627


RandomOverSampler
TP: 54
TN: 343
FP: 9
FN: 18
Accuracy： 0.9363207547169812
Recall： 0.75
Specificity： 0.9744318181818182
Precision： 0.8571428571428571
f1_score： 0.7999999999999999
AUC： 0.8622159090909092


RandomUnderSampler
TP: 58
TN: 312
FP: 40
FN: 14
Accuracy： 0.8726415094339622
Recall： 0.8055555555555556
Specificity： 0.8863636363636364
Precision： 0.5918367346938775
f1_score： 0.6823529411764706
AUC： 0.8459595959595959




### 資料集3

#### baseline

In [96]:
SVM_model(X_train_3,X_test_3,y_train_3,y_test_3)
LR_model(X_train_3,X_test_3,y_train_3,y_test_3)
RF_model(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model(X_train_3,X_test_3,y_train_3,y_test_3)

SVM baseline
TP: 44
TN: 350
FP: 2
FN: 28
Accuracy： 0.9292452830188679
Recall： 0.6111111111111112
Specificity： 0.9943181818181818
Precision： 0.9565217391304348
f1_score： 0.7457627118644068
AUC： 0.8027146464646465


LR baseline
TP: 17
TN: 352
FP: 0
FN: 55
Accuracy： 0.8702830188679245
Recall： 0.2361111111111111
Specificity： 1.0
Precision： 1.0
f1_score： 0.38202247191011235
AUC： 0.6180555555555556


RF baseline
TP: 0
TN: 352
FP: 0
FN: 72
Accuracy： 0.8301886792452831
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_24784/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 54
TN: 341
FP: 11
FN: 18
Accuracy： 0.9316037735849056
Recall： 0.75
Specificity： 0.96875
Precision： 0.8307692307692308
f1_score： 0.7883211678832116
AUC： 0.859375




#### 處理類別不平衡(SVM)

In [95]:
SVM_model2(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model3(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model4(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 47
TN: 342
FP: 10
FN: 25
Accuracy： 0.9174528301886793
Recall： 0.6527777777777778
Specificity： 0.9715909090909091
Precision： 0.8245614035087719
f1_score： 0.7286821705426356
AUC： 0.8121843434343434


SMOTE
TP: 46
TN: 342
FP: 10
FN: 26
Accuracy： 0.9150943396226415
Recall： 0.6388888888888888
Specificity： 0.9715909090909091
Precision： 0.8214285714285714
f1_score： 0.7187499999999999
AUC： 0.805239898989899


RandomOverSampler
TP: 45
TN: 347
FP: 5
FN: 27
Accuracy： 0.9245283018867925
Recall： 0.625
Specificity： 0.9857954545454546
Precision： 0.9
f1_score： 0.7377049180327869
AUC： 0.8053977272727273


RandomUnderSampler
TP: 53
TN: 298
FP: 54
FN: 19
Accuracy： 0.8278301886792453
Recall： 0.7361111111111112
Specificity： 0.8465909090909091
Precision： 0.4953271028037383
f1_score： 0.5921787709497207
AUC： 0.7913510101010102




#### 處理類別不平衡(Adaboost)

In [97]:
AdaBoost_model2(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model3(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model4(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 55
TN: 342
FP: 10
FN: 17
Accuracy： 0.9363207547169812
Recall： 0.7638888888888888
Specificity： 0.9715909090909091
Precision： 0.8461538461538461
f1_score： 0.802919708029197
AUC： 0.8677398989898989


SMOTE
TP: 56
TN: 340
FP: 12
FN: 16
Accuracy： 0.9339622641509434
Recall： 0.7777777777777778
Specificity： 0.9659090909090909
Precision： 0.8235294117647058
f1_score： 0.7999999999999999
AUC： 0.8718434343434344


RandomOverSampler
TP: 54
TN: 336
FP: 16
FN: 18
Accuracy： 0.9198113207547169
Recall： 0.75
Specificity： 0.9545454545454546
Precision： 0.7714285714285715
f1_score： 0.7605633802816902
AUC： 0.8522727272727273


RandomUnderSampler
TP: 60
TN: 301
FP: 51
FN: 12
Accuracy： 0.8514150943396226
Recall： 0.8333333333333334
Specificity： 0.8551136363636364
Precision： 0.5405405405405406
f1_score： 0.6557377049180328
AUC： 0.844223484848485


